In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase import build
from ase.optimize import FIRE
from ase.io import read
from agox.databases import Database
from agox.environments import Environment
from agox.utils.graph_sorting import Analysis

import numpy as np
from sklearn.decomposition import PCA
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

In [ ]:
## Set up the plotting environment
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rc('text', usetex=True)
plt.rc('font', family='cmr10', size=12)
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
## Set the plotting parameters
seed = 2
identifier = ""
output_directory_prefix = "DOutput"
# min_energy = -3.6635127# -3.7717605425

In [ ]:
## Set the descriptors
from agox.models.descriptors import SOAP, Voronoi
local_descriptor = local_descriptor = SOAP.from_species(["Si", "Ge"], r_cut=5.0)

graph_descriptor = Voronoi(
    covalent_bond_scale_factor=1.3, n_points=8, angle_from_central_atom=20, environment=None
)

In [ ]:
# Set the calculators
from chgnet.model import CHGNetCalculator
# from mace.calculators import mace_mp
from ase.calculators.singlepoint import SinglePointCalculator
calc = CHGNetCalculator()
# calc = mace_mp(model="../../mace-mpa-0-medium.model")

In [ ]:
## Load the unrelaxed structures
unrlxd_structures = read(output_directory_prefix+identifier+"/unrlxd_structures_seed"+str(seed)+".traj", index=":")
# for structure in unrlxd_structures1:
#   structure.calc = calc


In [ ]:
## Load the relaxed structures
rlxd_structures = read(output_directory_prefix+identifier+"/rlxd_structures_seed"+str(seed)+".traj", index=":")
# for structure in rlxd_structures:
#   structure.calc = calc

In [ ]:
# ## Read energies from energies_unrlxd_seed0.txt and add to the respective structures using a SinglePointCalculator
# ## The file has the form "index energy"
# ## This is done because there seem to be issues with storing the energy in the ASE trajectory file for some setups
# filename = "DOutput"+identifier+"/energies_unrlxd_seed"+str(seed)+".txt"
# with open(filename) as f:
#     for line in f:
#         index, energy = line.split()
#         index = int(index)
#         energy = float(energy)
#         unrlxd_structures[index].calc = SinglePointCalculator(unrlxd_structures[index], energy=energy * len(unrlxd_structures[index]))


# filename = "DOutput"+identifier+"/energies_rlxd_seed"+str(seed)+".txt"
# with open(filename) as f:
#     for line in f:
#         index, energy = line.split()
#         index = int(index)
#         energy = float(energy)
#         rlxd_structures[index].calc = SinglePointCalculator(rlxd_structures[index], energy=energy * len(rlxd_structures[index]))


In [ ]:
## Get bulk reference energies
Si_bulk = build.bulk("Si", crystalstructure="diamond", a=5.43)
Si_bulk.calc = calc
Si_reference_energy = Si_bulk.get_potential_energy() / len(Si_bulk)
Si_cubic = build.make_supercell(Si_bulk, [[-1, 1, 1], [1, -1, 1], [1, 1, -1]])

Ge_bulk = build.bulk("Ge", crystalstructure="diamond", a=5.65)
Ge_bulk.calc = calc
Ge_reference_energy = Ge_bulk.get_potential_energy() / len(Ge_bulk)
Ge_cubic = build.make_supercell(Ge_bulk, [[-1, 1, 1], [1, -1, 1], [1, 1, -1]])


In [ ]:
## Get slab energetics
Si_slab_vac = read("../Si_slab.vasp")
Ge_slab_vac = read("../Ge_slab.vasp")
Si_slab_vac.calc = calc
Ge_slab_vac.calc = calc
optimizer = FIRE(Si_slab_vac)
optimizer.run(fmax=0.05, steps=100)
optimizer = FIRE(Ge_slab_vac)
optimizer.run(fmax=0.05, steps=100)

In [ ]:
## Get abrupt interface energetics
SiGe_abrupt = read("../SiGe_abrupt_interface.vasp")
SiGe_abrupt.calc = calc
SiGe_abrupt.set_cell(rlxd_structures[0].get_cell(), scale_atoms=True)
optimizer = FIRE(SiGe_abrupt)
optimizer.run(fmax=0.05, steps=100)

In [ ]:
## Get abrupt interface area
area = np.linalg.norm(np.cross(SiGe_abrupt.get_cell()[0], SiGe_abrupt.get_cell()[1]))
print("Interface area: ", area)

In [ ]:
## Function to get the interface formation energy
def get_interface_energy(struc, Si_slab, Ge_slab, extra=False):
    energy = struc.get_potential_energy()
    cell = struc.get_cell()
    area = np.linalg.norm(np.cross(cell[0], cell[1]))
    Si_energy = Si_slab.get_potential_energy()
    Ge_energy = Ge_slab.get_potential_energy()
    ## need to subtract remaining silicon and germanium energies also
    if extra:
        return (energy - Si_energy - Ge_energy - 16*(Si_reference_energy + Ge_reference_energy)) / (2.0 * area)
    else:
        return (energy - Si_energy - Ge_energy) / (2.0 * area)

In [ ]:
# print("abrupt match 2 layers energy: ", get_interface_energy(abrupt, Si_slab, Ge_slab))
# print("abrupt match 3 layers energy: ", get_interface_energy(abrupt_3, Si_slab_3, Ge_slab_3))

In [ ]:
abrupt_en_per_area = get_interface_energy(SiGe_abrupt, Si_slab_vac, Ge_slab_vac, False)
print("Abrupt min energy: ", abrupt_en_per_area)

In [ ]:
## Calculate energies per atom for each unrelaxed structure
unrlxd_en_per_area = [get_interface_energy(structure, Si_slab_vac, Ge_slab_vac, False) for structure in unrlxd_structures]
print("Unrelaxed min energy: ", np.min(unrlxd_en_per_area))

In [ ]:
## Calculate energies per atom for each relaxed structure
rlxd_en_per_area = [get_interface_energy(structure, Si_slab_vac, Ge_slab_vac, False) for structure in rlxd_structures]
print("Relaxed min energy: ", np.min(rlxd_en_per_area))

In [ ]:
# print the min, max, mean, and std of the relaxed energies
print("Relaxed energies per atom: min = {:.2f}, max = {:.2f}, mean = {:.2f}, std = {:.2f}".format(
    np.min(rlxd_en_per_area), np.max(rlxd_en_per_area), 
    np.mean(rlxd_en_per_area), np.std(rlxd_en_per_area)))
# plot in latex table format
print("Si|Ge search & {:.2f} & {:.2f} & {:.2f} & {:.2f} \\\\".format(
    np.min(rlxd_en_per_area), np.max(rlxd_en_per_area), 
    np.mean(rlxd_en_per_area), np.std(rlxd_en_per_area)))

In [ ]:
## Set up the PCA
pca = PCA(n_components=2)

In [ ]:
## Get the 'super atom' descriptors for the unrelaxed structures
unrlxd_super_atoms = []
for structure in unrlxd_structures:
  unrlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
## Get the 'super atom' descriptors for the relaxed structures
rlxd_super_atoms = []
for structure in rlxd_structures:
  rlxd_super_atoms.append( np.mean(local_descriptor.get_features(structure), axis=0) )

In [ ]:
## Fit the PCA model to the unrelaxed or relaxed structures
rlxd_string = "rlxd"

In [ ]:
## Save pca model
import pickle
if False:
  pca.fit(np.squeeze([arr for arr in rlxd_super_atoms]))
  with open("pca_model_all_rlxd_"+str(seed)+".pkl", "wb") as f:
    pickle.dump(pca, f)

## Load pca model
with open("pca_model_all_"+rlxd_string+"_2.pkl", "rb") as f:
  pca = pickle.load(f)

In [ ]:
abrupt_super_atom = []
abrupt_super_atom.append(np.mean(local_descriptor.get_features(SiGe_abrupt), axis=0))
abrupt_super_atom.append(np.mean(local_descriptor.get_features(SiGe_abrupt), axis=0))

In [ ]:
## Transform the unrelaxed and relaxed structures to the reduced space
unrlxd_X_reduced = pca.transform(np.squeeze([arr for arr in unrlxd_super_atoms]))
rlxd_X_reduced = pca.transform(np.squeeze([arr for arr in rlxd_super_atoms]))
abrupt_X_reduced = pca.transform(np.squeeze([arr for arr in abrupt_super_atom]))

In [ ]:
## Get the index of the structure with the minimum energy
min_energy_index = np.argmin(rlxd_en_per_area)
print(min_energy_index)

In [ ]:
## Plot the PCA
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))

plt.subplots_adjust(wspace=0.05, hspace=0)

## Get the maximum energy for the colourbar
scaled_unrlxd_ens = [ ( x - abrupt_en_per_area ) * 1000 for x in unrlxd_en_per_area]
scaled_rlxd_ens = [ ( x - abrupt_en_per_area ) * 1000 for x in rlxd_en_per_area]
scaled_abrupt_en = abrupt_en_per_area * 1000
min_en = min(0.0, min(np.min(scaled_unrlxd_ens), np.min(scaled_rlxd_ens)))
max_en = min(max(np.max(scaled_unrlxd_ens), np.max(scaled_rlxd_ens)), 0.05 * 1000 + min_en)

## Plot the PCA
axes[0].scatter(unrlxd_X_reduced[:, 0], unrlxd_X_reduced[:, 1], c=scaled_unrlxd_ens, cmap="viridis", vmin = min_en, vmax = max_en)
axes[1].scatter(rlxd_X_reduced[:, 0], rlxd_X_reduced[:, 1], c=scaled_rlxd_ens, cmap="viridis", vmin = min_en, vmax = max_en)
axes[1].scatter(rlxd_X_reduced[min_energy_index, 0], rlxd_X_reduced[min_energy_index, 1], s=200, edgecolor=[1.0, 0.5, 0.5, 0.8], facecolor='none', linewidth=2, label="Intermixed")

## Add the minimum energy structures to the plot
for ax in axes:
    ax.scatter(abrupt_X_reduced[0, 0], abrupt_X_reduced[0, 1], s=200, edgecolor='red', facecolor='none', linewidth=2, label="Abrupt")
    # ax.scatter(abrupt_X_reduced[0, 0], abrupt_X_reduced[0, 1], c=scaled_abrupt_en, cmap="viridis", vmin = min_en, vmax = max_en)
    if ax == axes[1]:
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles[::-1], labels[::-1], facecolor='white', framealpha=1.0, edgecolor='black', fancybox=False, bbox_to_anchor=(1.0, 1.0), fontsize=20, handletextpad=0.2, borderpad=0.3, handlelength=1)
  
## Add labels
fig.text(0.5, 0.0, 'Principal component 1', ha='center', fontsize=20)
axes[0].set_ylabel('Principal component 2', fontsize=20)
axes[0].set_title('Unrelaxed', fontsize=20)
axes[1].set_title('Relaxed', fontsize=20)
if rlxd_string == "rlxd":
    xlims = [-4, 4]
    ylims = [-1, 3.5]
else:
    xlims = [-42, 55]
    ylims = [-12, 30]

for ax in axes:
    ax.tick_params(axis='both', direction='in', length=6, labelsize=20)
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.tick_params(axis='both', which='minor', length=3, direction='in')
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

## Unify tick labels
xticks = axes[0].get_xticks()
xticks = xticks[(xticks >= xlims[0]) & (xticks <= xlims[1])]

axes[1].set_xticks(xticks)
axes[1].set_yticklabels([])
axes[0].tick_params(axis='x', labelbottom=True, top=True)
axes[1].tick_params(axis='x', labelbottom=True, top=True)
axes[0].tick_params(axis='y', labelbottom=True, right=True)
axes[1].tick_params(axis='y', labelbottom=True, right=True)

## Make axes[0] and axes[1] the same width
axes[0].set_box_aspect(1.7)
axes[1].set_box_aspect(1.7)

## Add colorbar next to the axes
cbar = fig.colorbar(axes[1].collections[0], ax=axes, orientation='vertical', fraction=0.085, pad=0.02)
cbar.ax.tick_params(labelsize=20)
# cbar.ax.yaxis.set_major_locator(plt.FixedLocator([-61, -59, -57, -55, -53, -51]))
cbar.ax.yaxis.set_minor_locator(AutoMinorLocator(2))
cbar.set_label('Formation energy (meV/Å$^2$)', fontsize=20)

## Save the figure
plt.savefig('Si-Ge_RAFFLE'+identifier+'_pca_'+rlxd_string+'_fit_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')

In [ ]:
from ase.io import read, write
import numpy as np

atoms_mace = read("DResponse/rlxd_structures_seed2.traj", index=":")
atoms_chgnet = read("DResponse_CHGNet/rlxd_structures_seed0.traj", index=":")


In [ ]:
energies = [atoms_mace[i].get_potential_energy() for i in range(len(atoms_mace))]
sorted_indices = np.argsort(energies)
lowest_energy_structures = [atoms_mace[i] for i in sorted_indices[:2]]
# write the structures to a file
write("mace_1.vasp", lowest_energy_structures[0], sort=True, direct=True)
write("mace_2.vasp", lowest_energy_structures[1], sort=True, direct=True)

In [ ]:
energies = [atoms_chgnet[i].get_potential_energy() for i in range(len(atoms_chgnet))]
sorted_indices = np.argsort(energies)
lowest_energy_structures = [atoms_chgnet[i] for i in sorted_indices[:2]]
# write the structures to a file
write("chgnet_1.vasp", lowest_energy_structures[0], sort=True, direct=True)
write("chgnet_2.vasp", lowest_energy_structures[1], sort=True, direct=True)

In [ ]:
SiGe_abrupt = read("../SiGe_abrupt_interface.vasp")
SiGe_abrupt.set_cell(atoms_mace[0].get_cell(), scale_atoms=True)
write("SiGe_abrupt_interface_rescaled.vasp", SiGe_abrupt, sort=True, direct=True)

In [ ]:
from raffle.generator import raffle_generator

generator = raffle_generator()
generator.distributions.read_gdfs( output_directory_prefix+identifier+"/gdfs_seed"+str(seed)+".txt" )
descriptor = generator.get_descriptor()

# Create a figure with 3 subplots side by side
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot for each n-body descriptor (2-body, 3-body, 4-body)
colours = [ 'black', 'red', 'blue' ]
labels_2body = [ 'Si-Si', 'Si-Ge', 'Ge-Ge' ]
labels_3body = [ 'Si', 'Ge' ]
for j in range(3):
    # Calculate x-axis values
    x = np.arange(generator.distributions.cutoff_min[j],
                generator.distributions.cutoff_max[j] + generator.distributions.width[j],
                generator.distributions.width[j])

    if j == 0:
        labels = labels_2body
    else:
        labels = labels_3body
    # Plot on the respective subplot
    for idx in range(len(descriptor[j])):
        axes[j].plot(x, descriptor[j][idx,:],
                     color=colours[idx],
                    label=labels[idx],
        )

    # Set labels and title for each subplot
    # axes[j].set_ylabel('Descriptor value')
    # axes[j].set_title(f'{j+2}-body descriptor')

# set the legend font size
for ax in axes:
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(16)
    ax.legend(fontsize=16, loc='upper right', framealpha=0.5, edgecolor='black', facecolor='white', handletextpad=0.1)

axes[0].set_ylabel('Distribution function (arb. units)', fontsize=20)
axes[0].set_xlabel('Bond length (Å)', fontsize=20)
axes[1].set_xlabel('3-body angle (radians)', fontsize=20)
axes[2].set_xlabel('4-body angle (radians)', fontsize=20)
axes[0].set_xlim(0, 6)
axes[1].set_xlim(0, np.pi)
axes[2].set_xlim(0, np.pi)

axes[0].set_ylim(0, None)
axes[1].set_ylim(0, 1.0)
axes[2].set_ylim(0, 1.0)

# reduce number of ticks to five at most
axes[0].xaxis.set_major_locator(plt.MaxNLocator(3))
axes[0].yaxis.set_major_locator(plt.MaxNLocator(3))

# set x ticks to 0, 1, 2, 3
axes[1].set_xticks([0, 1, 2, 3])
axes[2].set_xticks([0, 1, 2, 3])
# axes[1].set_yticks([0, 0.1, 0.2])
# axes[2].set_yticks([0, 0.05, 0.1])

# have the ticks point intwards and on both sides
for ax in axes:
    ax.tick_params(axis='both', which='major', direction='in', length=10, width=1)
    ax.tick_params(axis='both', which='minor', direction='in', length=5, width=1)
    ax.tick_params(axis='x', which='both', bottom=True, top=True)
    ax.tick_params(axis='y', which='both', left=True, right=True)

axes[1].yaxis.set_major_locator(MultipleLocator(0.5))
axes[2].yaxis.set_major_locator(MultipleLocator(0.5))
# add minor ticks to all axes
for ax in axes:
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    


# axes[0].set_xlabel('Distance (Å)')
# axes[1].set_xlabel('3-body angle (radians)')
# axes[2].set_xlabel('Improper dihedral angle (radians)')
# plt.tight_layout()
# plt.show()

# get figures closer together
plt.subplots_adjust(wspace=0.1, hspace=0)
# remove labels for y axis for the second and third plots

plt.savefig('Si-Ge_RAFFLE'+identifier+'_gen_descriptor_rlxd_seed'+str(seed)+'.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')
